 - 네이버 영화평 감성지수 분석

In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.5 MB/s 
     |████████████████████████████████| 448 kB 36.3 MB/s 


In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [53]:
# 네이버 영화 리뷰 데이터로 검색
train_df = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', sep='\t')
test_df = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', sep='\t')

In [54]:
print(train_df.shape, test_df.shape)
train_df.head(3)

(150000, 3) (50000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


### 1. 데이터 전처리

 - 훈련 데이터 셋

In [55]:
# Null 데이터가 있는지 확인
train_df.isna().sum()

id          0
document    5
label       0
dtype: int64

In [56]:
# Null 데이터 제거
train_df.dropna(how='any', inplace=True) 
train_df.shape

(149995, 3)

In [57]:
# 중복여부 확인
train_df.document.nunique()

146182

In [58]:
# 중복 데이터 제거
train_df.drop_duplicates(subset=['document'], inplace=True) 
train_df.shape

(146182, 3)

In [59]:
# 긍정(1)/부정(0) 분포
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

 - 테스트 데이터셋

In [60]:
test_df.isna().sum()

id          0
document    3
label       0
dtype: int64

In [61]:
# NaN 데이터 제거
test_df.dropna(how='any', inplace=True)
test_df.shape

(49997, 3)

In [62]:
test_df.document.nunique()

49157

In [63]:
# 중복 데이터 제거
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape

(49157, 3)

In [64]:
# 긍정(1)  / 부정(0) 분포
test_df.label.value_counts()

1    24711
0    24446
Name: label, dtype: int64

### 2. 텍스트 전처리
 - Train datasets

In [65]:
# 한글 이외의 문자는 공백으로 처리하고 strip
train_df.document = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ').str.strip()
train_df.head(3)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [66]:
# '' 만 남은 데이터 --> np.nan 으로 대체한 후 제거
train_df.document.replace('', np.nan, inplace=True)
train_df.document.isna().sum()

789

In [67]:
train_df.dropna(how='any', inplace=True)
train_df.shape

(145393, 3)

 - test 데이터 셋

In [25]:
# 한글 이외 문자 공백 처리하고 strip()
test_df.document = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ').str.strip()
test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,,0
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0


In [68]:
# ' ' 만 남은 데이터 --> np.nan 대체 후 제거
test_df.document = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ').str.strip()
test_df.document.replace('', np.nan, inplace=True)
test_df.dropna(how='any', inplace=True)
test_df.shape

(48852, 3)

 - 전처리가 끝난 데이터 저장

In [69]:
train_df.to_csv('naver_movie_train_전처리완료.tsv', sep='\t', index=False)
test_df.to_csv('naver_movie_test_전처리완료.tsv', sep='\t', index=False)

### 3. 한글처리

In [70]:
from konlpy.tag import Okt 
okt = Okt()

In [71]:
text = '교도소 이야기구먼 솔직히 재미는 없다평점 조정'
okt.morphs(text)

['교도소', '이야기', '구먼', '솔직히', '재미', '는', '없다', '평점', '조정']

In [72]:
okt.morphs(text, stem=True)

['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']

In [73]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을','ㅋㅋ','ㅠㅠ','ㅎㅎ', '에게', '에', '구만', '구먼']

In [74]:
' '.join([word for word in okt.morphs(text, stem=True) if word not in stopwords])

'교도소 이야기 솔직하다 재미 없다 평점 조정'

In [75]:
from tqdm.notebook import tqdm

X_train = []
for sentence in tqdm(train_df.document):
    morphs = okt.morphs(sentence, stem=True)
    tmp_str = ' '.join([word for word in morphs if word not in stopwords])
    X_train.append(tmp_str)

  0%|          | 0/145393 [00:00<?, ?it/s]

In [76]:
X_test = []

for sentence in tqdm(test_df.document):
    morphs = okt.morphs(sentence, stem= True)
    tmp_str =  ' '.join([word for word in morphs if word not in stopwords])
    X_test.append(tmp_str)

  0%|          | 0/48852 [00:00<?, ?it/s]

In [77]:
y_train = train_df.label.values 
y_test = test_df.label.values

### 4. Feature 변환 모델 학습

In [78]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [79]:
cv = CountVectorizer()
lr = LogisticRegression(random_state=2022)
pipeline = Pipeline([("CVECT", cv), ('LR', lr)])
%time pipeline.fit(X_train, y_train)

CPU times: user 8.29 s, sys: 5.88 s, total: 14.2 s
Wall time: 8.38 s


Pipeline(steps=[('CVECT', CountVectorizer()),
                ('LR', LogisticRegression(random_state=2022))])

In [80]:
pipeline.score(X_test, y_test)

0.8275403258822566

### 5. 실제 데이터 적용

In [81]:
rv1 = '모든 국민이 꼭 봤으면 하는 영화입니다.'
rv2 = '생각보다 별로였네요... 보면서 졸았습니다.'

In [82]:
import re
rv1 = re.sub('[^가-힣]', ' ', rv1)
rv2 = re.sub('[^가-힣]', ' ', rv2)

In [83]:
morphs = okt.morphs(rv1, stem= True)
rv1 =  ' '.join([word for word in morphs if word not in stopwords])
morphs = okt.morphs(rv2, stem= True)
rv2 =  ' '.join([word for word in morphs if word not in stopwords])

In [85]:
pipeline.predict([rv1, rv2])

array([1, 0])

### 6. 하이퍼 파라미터 튜닝

In [86]:
from sklearn.model_selection import GridSearchCV
params = {'CVECT__ngram_range': [(1,1),(1,2)], 'CVECT__max_df': [0.95, 0.98], 'LR__C': [1, 5]}

In [87]:
grid_pipe = GridSearchCV(pipeline, params, scoring='accuracy', cv=3, n_jobs=-1)
%time grid_pipe.fit(X_train, y_train)

CPU times: user 1min 8s, sys: 21.8 s, total: 1min 30s
Wall time: 4min 37s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('CVECT', CountVectorizer()),
                                       ('LR',
                                        LogisticRegression(random_state=2022))]),
             n_jobs=-1,
             param_grid={'CVECT__max_df': [0.95, 0.98],
                         'CVECT__ngram_range': [(1, 1), (1, 2)],
                         'LR__C': [1, 5]},
             scoring='accuracy')

In [88]:
grid_pipe.best_params_

{'CVECT__max_df': 0.95, 'CVECT__ngram_range': (1, 2), 'LR__C': 1}

In [89]:
grid_pipe.best_estimator_.score(X_test, y_test)

0.8478874969295014